In [30]:
from mido import MidiFile
import time
import operator
from mido import MidiFile
piece = "data/hand_picked_spotify/15/midsummer_10_orch.mid"
mid_in = MidiFile(piece,clip=True)

In [33]:
from miditoolkit.midi import parser as mid_parser  
from miditoolkit.midi import containers as ct
mido_obj = mid_parser.MidiFile(piece)
mido_obj.ticks_per_beat

384

NameError: name 'noteMidi' is not defined

In [ ]:
for channel in mid_in.tracks[5:]:
    # print(channel)
    break

In [ ]:
from music21 import *
c = converter.parse(piece) #Also OK, check how to process

In [ ]:
#https://stackoverflow.com/questions/63105201/python-mido-how-to-get-note-starttime-stoptime-track-in-a-list
#https://dev.to/varlen/editing-midi-files-with-python-2m0g (checking how to get back this)

In [42]:
import numpy as np
class noteMidi:
    def __init__(self,p,s,e):
        self.pitch = p
        self.onset = s
        self.offset = e
        
def mergeIntervals(arr):
        # Sorting based on the increasing order 
        # of the start intervals
        arr.sort(key = lambda x: x[0]) 
        # array to hold the merged intervals
        m = []
        s = -10000
        max = -100000
        for i in range(len(arr)):
            a = arr[i]
            if a[0] > max:
                if i != 0:
                    m.append([s,max])
                max = a[1]
                s = a[0]
            else:
                if a[1] >= max:
                    max = a[1]        
        #'max' value gives the last point of 
        # that particular interval
        # 's' gives the starting point of that interval
        # 'm' array contains the list of all merged intervals
        if max != -100000 and [s, max] not in m:
            m.append([s, max])
        return m

def gettop(note,intervals):
    note_interval = [note.onset,note.offset]
    overlap_time = 0
    total_time = note.offset - note.onset
    if total_time == 0:
        return 1 #(we do not need this note)
    for interval in intervals:
        maxstart = max(note_interval[0],interval[0])
        minend = min(note_interval[1],interval[1])
        if maxstart < minend:
            overlap_time += minend-maxstart
    return overlap_time/total_time

def skyline(notes): #revised skyline algorithm by Chai, 2000
    #Performed on a single channel
    accepted_notes = []
    notes = sorted(notes, key=lambda x: x.pitch, reverse=True)
    intervals = []
    for note in notes:
        if gettop(note,intervals) <=0.5:
            accepted_notes.append(note)
            intervals.append([note.onset,note.offset])
            intervals = mergeIntervals(intervals)
    return sorted(accepted_notes,key=lambda x: x.onset)
    
    
            

In [43]:
channel_no = 6
notelist = []
for inst mido_obj.instruments:
    for notes in inst.notes:
        notelist.append(noteMidi(notes.pitch,notes.start,notes.end))
notelist_new = skyline(notelist)
print(len(notelist),len(notelist_new))

152 76


In [48]:
mido_out = mid_parser.MidiFile()
mido_out.ticks_per_beat = 384
print(mido_out)
beat_resol = mido_out.ticks_per_beat
track = ct.Instrument(program=0,is_drum=False,name='example track')
mido_out.instruments = [track]
for note in notelist_new:
    mido_out.instruments[0].notes.append(ct.Note(start=note.onset,end=note.offset,pitch=note.pitch,velocity=30))
mido_out.dump("result.mid")

ticks per beat: 384
max tick: 0
tempo changes: 0
time sig: 0
key sig: 0
markers: 0
lyrics: False
instruments: 0
result.mid


In [ ]:
mididict = []
output = []
# print(mid_in)
# Put all note on/off in midinote as dictionary.
for i in mid_in:
    if i.type == 'note_on' or i.type == 'note_off' or i.type == 'time_signature':
        mididict.append(i.dict())
# change time values from delta to relative time.
mem1=0
for i in mididict:
    time = i['time'] + mem1
    i['time'] = time
    mem1 = i['time']
# make every note_on with 0 velocity note_off
    if i['type'] == 'note_on' and i['velocity'] == 0:
        i['type'] = 'note_off'
# put note, starttime, stoptime, as nested list in a list. # format is [type, note, time, channel]
    mem2=[]
    if i['type'] == 'note_on' or i['type'] == 'note_off':
        mem2.append(i['type'])
        mem2.append(i['note'])
        mem2.append(i['time'])
        mem2.append(i['channel'])
        output.append(mem2)
# put timesignatures
    if i['type'] == 'time_signature':
        mem2.append(i['type'])
        mem2.append(i['numerator'])
        mem2.append(i['denominator'])
        mem2.append(i['time'])
        output.append(mem2)
# viewing the midimessages.
for i in output:
    if i[3] == 13:
        print(i)
print(mid_in.ticks_per_beat)

['note_on', 69, 0.3240265, 13]
['note_off', 69, 0.495965439453125, 13]
['note_on', 74, 0.569928, 13]
['note_off', 74, 0.741866939453125, 13]
['note_on', 78, 0.8158295, 13]
['note_off', 78, 0.987768439453125, 13]
['note_on', 78, 1.3076325, 13]
['note_off', 78, 1.479571439453125, 13]
['note_on', 74, 1.553534, 13]
['note_off', 74, 1.725472939453125, 13]
['note_on', 69, 1.7994355, 13]
['note_off', 69, 1.971374439453125, 13]
['note_on', 69, 2.2912385, 13]
['note_off', 69, 2.463177439453125, 13]
['note_on', 74, 2.53714, 13]
['note_off', 74, 2.709078939453125, 13]
['note_on', 78, 2.7830415, 13]
['note_off', 78, 2.954980439453125, 13]
['note_on', 69, 3.2748445, 13]
['note_off', 69, 3.446783439453125, 13]
['note_on', 74, 3.520746, 13]
['note_off', 74, 3.692684939453125, 13]
['note_on', 78, 3.7666475, 13]
['note_off', 78, 3.938586439453125, 13]
['note_on', 78, 4.2584505, 13]
['note_off', 78, 4.430389439453125, 13]
['note_on', 74, 4.504352000000001, 13]
['note_off', 74, 4.676290939453126, 13]
['n